# Pitch Control

![](figures/PID_en.svg)

* r --> desired
* y --> actual pitch
* u --> duty

In [1]:
%connect serial:///dev/ttyAMA1

Connected to robot-stm32 @ serial:///dev/ttyAMA1


PID controller:

In [4]:
%%writefile $IOT_PROJECTS/robot/code/stm32/lib/PID.py
# Based on https://github.com/br3ttb/Arduino-PID-Library

from array import array

# config vector (may be dynamically updated)
SETPOINT = const(0)  # setpoint            
KP       = const(1)  # proportional term
KI       = const(2)  # scaled by /fs
KD       = const(3)  # scaled by *fs
U_MIN    = const(4)  # minimum PID output (anti-windup)
U_MAX    = const(5)  # maximum PID output (anti-windup)

# state (used internally)
_SUM     = const(0)
_Y       = const(1)

class PID:
    
    def __init__(self, config):
        self.config = config
        self.state = array('f', [0, 0])
    
    def update(self, y):
        """compute & return new PID output u from plant output y"""
        c = self.config
        s = self.state
        err = c[SETPOINT] - y
        s[_SUM] += self._clip(c[KI] * err)   # integrator state
        u = self._clip(c[KP] * err + s[_SUM] - c[KD] * (y - s[_Y]))
        s[_Y] = y   # save last y (for KD term)
        return u
    
    def _clip(self, value):
        c = self.config
        if value > c[U_MAX]: return c[U_MAX]
        if value < c[U_MIN]: return c[U_MIN]
        return value

Writing /home/iot/iot49.org/docs/projects/robot/code/stm32/lib/PID.py


Pitch controller:

In [3]:
%%writefile $IOT_PROJECTS/robot/code/stm32/lib/pitch_control.py
from controller import *
from param import PARAM, PARAM_RESERVED
from pid import PID


P0 = const(2)
assert P0 == PARAM_RESERVED

SET_PITCH = const(P0+0)
KP        = const(P0+1)
KI        = const(P0+2)
KD        = const(P0+3)
U_MIN     = const(P0+4)
U_MAX     = const(P0+5)

class Control(Controller):
    
    def __init__(self, uart):
        super().__init__(uart)
        self.pid = PID(memoryview(PARAM)[SET_PITCH:U_MAX+1])
    
    def update(self):
        state = self.state
        pitch = state[STATE_PITCH]
        if abs(pitch) < 10:
            duty = self.fix_duty(self.pid.update(pitch))
        else:
            duty = 0
            state[STATE_DUTY1] = state[STATE_DUTY2] = duty
        
    @staticmethod
    def fix_duty(duty):
        if abs(duty) > 1:
            duty = duty+8.5 if duty>0 else duty-8.5
        return duty

Writing /home/iot/iot49.org/docs/projects/robot/code/stm32/lib/pitch_control.py


Run the show ...

In [3]:
%%host

import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

from asyncio_mqtt import Client
from struct import pack
import numpy as np
import asyncio, json, os

import stm32
from robot import *

print(f"plot @ http://{os.getenv('DNS_NAME')}.local:5006")

MQTT_BROKER = os.getenv("HOST_IP")
TOPIC_ROOT  = "public/vis"

P0          = PARAM_RESERVED

SET_PITCH   = const(P0+0)
KP          = const(P0+1)
KI          = const(P0+2)
KD          = const(P0+3)
U_MIN       = const(P0+4)
U_MAX       = const(P0+5)

FS          = 100        # controller update rate


class Control:

    def __init__(self, controller):
        self.controller = controller
        self.pitch = 90

    async def main(self):
        async with Client(MQTT_BROKER) as client, \
            Comm(self.state_listener) as robot:
            self.client = client
            await client.publish(f"{TOPIC_ROOT}/new", json.dumps({
                "columns": [ "time [s]", "pitch", "duty" ],
                "rollover": 1000,
                "args": { "title": f"Robot {self.controller}" },
            }))
            await robot.set(PARAM_FS, FS)
            print("starting ..., fs =", await robot.get(PARAM_FS))
            await robot.start(self.controller)
            print("started!")
            # wait for upright position
            while abs(self.pitch) > 3:
                print(f"orient robot upright, pitch {self.pitch} --> 0")
                await asyncio.sleep(1)
            # balancing ...
            while abs(self.pitch) <= 10:
                print(f"balancing, pitch = {self.pitch:8.2f}")
                await asyncio.sleep(1)
            print("main DONE")

    async def state_listener(self, state):
        try:
            t = state[STATE_K]/FS
            duty = state[STATE_DUTY1]
            self.pitch = state[STATE_PITCH]
            await self.client.publish(f"{TOPIC_ROOT}/bin", 
                                      pack('!3f', t, self.pitch, duty))
        except Exception as e:
            print("***** state_listener:", e)
            
    async def kp(self, value):
        await robot.set(PARAM_KP, value)
        
    async def ki(self, value):
        await robot.set(PARAM_KI, value/FS)
        
    async def kd(self, value):
        await robot.set(PARAM_KD, value*FS)

stm32.rsync()

c = Control("pitch_control")
asyncio.run(c.main())

plot @ http://pi4robot.local:5006
UPDATE  /flash/lib/pitch_control.py
MCU: >
     MicroPython v1.17-195-gbb7aae557 on 2022-01-09; Motor Hat with STM32F405RG
     Type "help()" for 
reset stm32
*** MCU: start Comm @ 1000000 baud

starting ..., fs = 100.0
started!
orient robot upright, pitch 90 --> 0
orient robot upright, pitch -85.60543823242188 --> 0
orient robot upright, pitch -85.60543823242188 --> 0
orient robot upright, pitch -85.60543823242188 --> 0
orient robot upright, pitch -85.60543823242188 --> 0
orient robot upright, pitch -85.60543823242188 --> 0
orient robot upright, pitch -85.60543823242188 --> 0
orient robot upright, pitch -85.60543823242188 --> 0
orient robot upright, pitch -85.00543212890625 --> 0
orient robot upright, pitch -75.95443725585938 --> 0
orient robot upright, pitch -67.16485595703125 --> 0
orient robot upright, pitch -28.488067626953125 --> 0
orient robot upright, pitch -7.919513702392578 --> 0
balancing, pitch =    -2.15
balancing, pitch =     1.23
balanci